In [1]:
# imports
import scipy.io
import numpy as np               # for arrays
from numpy import linalg as LA   # for eigenvalues
import matplotlib                # for plots
import time                      # for time measurements
from PIL import Image            # for showing images
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sympy

In [2]:
def show_img(img):
    temp = img.copy()
    temp.resize((46,56))
    im = Image.fromarray(temp.T)
    im.show()

Get training/test splits

In [3]:
# load data
mat = scipy.io.loadmat('face.mat')
raw_data = mat['X']

raw_data = np.transpose(raw_data)
N,D = raw_data.shape
C = 52 # number of classes in dataset
train_size = int(N * 0.8)
test_size = int(N * 0.2)

pca_training_data = np.empty([int(520*0.8), 2576])
pca_testing_data = np.empty([int(520*0.2), 2576])
lda_training_data = []
lda_testing_data = []

# create training and test data
for x in range(52):
    # 8/2 ratio for training and testing datasets
    lda_training_data.append(raw_data[x*10:x*10+8].copy())
    lda_testing_data.append(raw_data[x*10+8:(x+1)*10].copy())
    

lda_training_data = np.array(lda_training_data)
lda_testing_data = np.array(lda_testing_data)
pca_training_data = lda_training_data.reshape(train_size, D)
pca_testing_data = lda_testing_data.reshape(test_size,D)


In [4]:
print(lda_training_data.shape)

(52, 8, 2576)


In [5]:
# accepts list of data points 
def get_mean(data, axis):
    # return average of data points
    return data.mean(axis=axis)

We need to make sure that the generalised eigenvalue problem that we encounter when doing LDA is solvable by making sure that the within-class scatter matrix is non-singular.
We do this by first reducing the dimension of the data via PCA to an M <= N - c.

In [6]:
def get_Wpca(data):
    # copy in PCA code that gives the matrix of principal components
    return np.eye(data.shape[0])
    

We want a projection that maximises the ratio between the between-class scatter matrix and the within class scatter matrix.
The projection W turns out to be the solutions to the generalised eigen value problem. (Found via solving the langrangian. Slide 10-11)

In [24]:
Wpca = get_Wpca(pca_training_data)
mean_all_data = get_mean(pca_training_data.T, 1).T
mean_class_data = get_mean(lda_training_data, 1)
diff_class_mean = mean_class_data - mean_all_data
print(mean_class_data.shape)

# within class scatter (scalar)
# Sw = is an sum over the classes of the cov of each class and the respective means
# between class scatter (scalar)
Sb = np.dot(mean_class_data.T, mean_class_data)


# lda_evals, lda_evecs = LA.eig(np.dot(LA.inv(Sw),Sb))


(52, 2576)


In [23]:
print(Sb.shape)

(2576, 2576)
